# Damiano Bressanin 138075


Ho preso spunto da:
- "Very Deep Convolutional Networks for Large-Scale Image Recognition" https://arxiv.org/pdf/1409.1556.pdf
- "How to Classify Photos of Dogs and Cats" di Jason Brownlee. https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/
- https://keras.io/guides/transfer_learning/
- https://keras.io/api/applications/vgg/#vgg16-function



# Test:
- Dataset 80-10-10;
- Foto a colori;
- Sì Data Augmentation;
- Transfer Learning e Fine Tuning con VGG16;

In [1]:
import os
import shutil
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import numpy as np

from matplotlib.image import imread
from matplotlib import pyplot

random.seed(1)

2023-08-27 18:15:21.718623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing e Data Augmentation

In [2]:
from keras.preprocessing.image import ImageDataGenerator

# vedi paper

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    shear_range=0.2,
#    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.8,1.2],
#    rotation_range=20,
#    width_shift_range=0.15,
#    height_shift_range=0.15,
    fill_mode='nearest'
)
train_datagen.mean = [123.68, 116.779, 103.939]


test_datagen = ImageDataGenerator(featurewise_center=True)
test_datagen.mean = [123.68, 116.779, 103.939]

val_datagen = ImageDataGenerator(featurewise_center=True)
val_datagen.mean = [123.68, 116.779, 103.939]

batch_size = 32

train_it = train_datagen.flow_from_directory("dataset_diviso_v2/train/",
                                             class_mode='binary',
                                             batch_size=batch_size,
                                             target_size=(224, 224),
                                             color_mode="rgb",
                                             shuffle=True,
                                             seed=42
                                            )

val_it = val_datagen.flow_from_directory('dataset_diviso_v2/valid/',
                                         class_mode='binary',
                                         batch_size=batch_size,
                                         target_size=(224, 224),
                                         color_mode="rgb",
                                         shuffle=True,
                                         seed=42)

test_it = test_datagen.flow_from_directory('dataset_diviso_v2/test/',
                                           class_mode='binary',
                                           batch_size=batch_size,
                                           target_size=(224, 224),
                                           color_mode="rgb",
                                           shuffle=True,
                                           seed=42)



Found 19997 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


# Creazione del modello:
# Transfer Learning

In [3]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

def define_model():
    # carico il modello
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # congelo gli strati
    for layer in model.layers:
        layer.trainable = False
    # aggiungo parte finale
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(256, activation='relu', kernel_initializer='he_uniform')(flat1)
    class2 = Dense(256, activation='relu', kernel_initializer='he_uniform') (class1)
    output = Dense(1, activation='sigmoid')(class2)
    
    model = Model(inputs=model.inputs, outputs=output)
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [5]:
import datetime

unique_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + unique_name

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch')

In [6]:
from tensorflow.keras.optimizers import SGD

model=define_model()

history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=val_it, validation_steps=len(val_it), epochs=30, verbose=1, callbacks=[tensorboard_callback,early_stop],validation_split=0)

2023-08-27 18:15:26.389798: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 18:15:26.396457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 18:15:26.396838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 18:15:26.398457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 18:15:26.398970: I tensorflow/compile

Epoch 1/30


2023-08-27 18:15:30.517369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-27 18:15:35.949774: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1bb7c830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-27 18:15:35.949839: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-08-27 18:15:36.019037: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


370/625 [================>.............] - ETA: 1:44 - loss: 0.4380 - accuracy: 0.9514

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 [==============================] - 277s 432ms/step - loss: 0.2940 - accuracy: 0.9603 - val_loss: 0.0700 - val_accuracy: 0.9728
Epoch 2/30
625/625 [==============================] - 264s 421ms/step - loss: 0.0525 - accuracy: 0.9819 - val_loss: 0.0622 - val_accuracy: 0.9764
Epoch 3/30
625/625 [==============================] - 262s 418ms/step - loss: 0.0289 - accuracy: 0.9893 - val_loss: 0.0602 - val_accuracy: 0.9784
Epoch 4/30
625/625 [==============================] - 264s 421ms/step - loss: 0.0201 - accuracy: 0.9943 - val_loss: 0.0732 - val_accuracy: 0.9804
Epoch 5/30
625/625 [==============================] - 262s 419ms/step - loss: 0.0127 - accuracy: 0.9959 - val_loss: 0.0748 - val_accuracy: 0.9808
Epoch 6/30
625/625 [==============================] - 262s 420ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.0817 - val_accuracy: 0.9792
Epoch 7/30
625/625 [==============================] - 262s 419ms/step - loss: 0.0042 - accuracy: 0.9988 - val_loss: 0.0875 - val_accura

In [7]:
# valuto il modello
_, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

79/79 [==============================] - 11s 134ms/step - loss: 0.0511 - accuracy: 0.9828
> 98.280


# Fine Tuning

In [8]:
for layer in model.layers:
    layer.trainable = True

# learning rate più basso
opt=SGD(learning_rate=0.00005, momentum=0.9)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])



In [9]:
train_it.reset
val_it.reset
test_it.reset


history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=val_it, validation_steps=len(val_it), epochs=30, verbose=1, callbacks=[tensorboard_callback,early_stop],validation_split=0)


Epoch 1/30
625/625 [==============================] - 301s 465ms/step - loss: 0.0154 - accuracy: 0.9947 - val_loss: 0.0555 - val_accuracy: 0.9792
Epoch 2/30
625/625 [==============================] - 306s 488ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 0.0578 - val_accuracy: 0.9824
Epoch 3/30
625/625 [==============================] - 293s 469ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 0.0547 - val_accuracy: 0.9824
Epoch 4/30
625/625 [==============================] - 311s 497ms/step - loss: 0.0051 - accuracy: 0.9989 - val_loss: 0.0558 - val_accuracy: 0.9832
Epoch 5/30
625/625 [==============================] - 295s 471ms/step - loss: 0.0042 - accuracy: 0.9992 - val_loss: 0.0582 - val_accuracy: 0.9836
Epoch 6/30
625/625 [==============================] - 299s 477ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0591 - val_accuracy: 0.9840
Epoch 7/30
625/625 [==============================] - 315s 504ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.0619 -

In [10]:
# valuto il modello
_, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
print('> %.3f' % (acc * 100.0))

79/79 [==============================] - 11s 134ms/step - loss: 0.0439 - accuracy: 0.9880
> 98.800


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model_dir = "modelli/"+ unique_name +".keras"
model.save(model_dir)

In [13]:
def summarize_diagnostics(history):
    pyplot.figure(figsize=(20, 15))
    
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='orange', label='train')
    pyplot.plot(history.history['val_loss'], color='blue', label='val')
    pyplot.legend(loc='upper right') # legenda
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')  # griglia nera
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='red', label='train')
    pyplot.plot(history.history['val_accuracy'], color='green', label='val')
    pyplot.legend(loc='lower right')
    pyplot.grid(which='both', linestyle='-', linewidth=1, color='black')

    # salvo i plot su file
    filename = 'plot/' + unique_name
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
    
summarize_diagnostics(history)